# Imports & Connexion GG Drive


In [103]:
### Install BERTopic permanently in GG Drive : https://stackoverflow.com/questions/55253498/how-do-i-install-a-library-permanently-in-colab
"""import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

!pip install --target=$nb_path BERTopic"""

"import os, sys\nfrom google.colab import drive\ndrive.mount('/content/mnt')\nnb_path = '/content/notebooks'\nos.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)\nsys.path.insert(0, nb_path)  # or append(nb_path)\n\n!pip install --target=$nb_path BERTopic"

In [1]:
!pip install BERTopic

In [2]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
path_to_GGDrive = "/content/gdrive/MyDrive/"
print("-> Connexion à Google Drive OK")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
-> Connexion à Google Drive OK


# Chargement des données

In [4]:
def get_dataframe(file_name,lower=False,to_spaces=[]):
    """ Charge la base de données dans une dataframe python en effectuant un pré-traitement de base :
        - suppression des lignes vides
        - suppression des sauts de lignes et indentations dans les resumes 
        'to_spaces' permet de préciser quels autres séquences doivent être remplacées par des espaces dans le texte."""
    input_df = pd.read_excel(file_name)
    df = input_df[["EAN","Description sans html"]]
    df.columns = ['EAN','textes']
    # Suppression des lignes vides
    df = df.dropna().reset_index(drop=True)
    # Suppression des sauts de lignes et indentations dans les resumes
    for k in range(0,df.shape[0]):
        for elt in to_spaces:
            df.iloc[k,1] = df.iloc[k,1].replace(elt," ")
        if lower:
            df.iloc[k,1] = df.iloc[k,1].lower()
    print('Dataframe with shape {1} loaded from "{0}"\n'.format(file_name,df.shape))
    return(df)

In [5]:
data = get_dataframe(path_to_GGDrive + 'corpus 3.xlsx', lower=True, to_spaces=["\n","\t","&nbsp;"])
print("# CORPUS : ") ; print(data['textes'].head())

docs = data["textes"].to_list()

print("\nNombre de textes : ",len(docs))

Dataframe with shape (9086, 2) loaded from "/content/gdrive/MyDrive/corpus 3.xlsx"

# CORPUS : 
0    si nous connaissons pierre-gilles de gennes de...
1    exploration des mondes sous-marins, conquêtes ...
2    langage parlé, cinéma, chanson, publicité et, ...
3    dans la cellule de la prison où l'avait enferm...
4    siyu cao est d’abord un regard. un regard brid...
Name: textes, dtype: object

Nombre de textes :  9086


# Algorithme BERTopic

## Réaliser un Topic Modeling sur les données

#### Transformer CamemBERT

In [ ]:
!pip install flair

In [ ]:
from flair.embeddings import TransformerDocumentEmbeddings
camembert = TransformerDocumentEmbeddings('camembert-base')
topic_model = BERTopic(embedding_model=camembert)

#### Transformer BERT

In [ ]:
topic_model = BERTopic(language="french", verbose=True)

In [53]:
topic_model = BERTopic(language="multilingual", verbose=True)#, nr_topics="auto") # nr_topics : impose un nombre de topics (89 topics sur #3 contre 150 sans)

#### Execution de BERTopic (2/3 min)

In [54]:
topics, probabilities = topic_model.fit_transform(docs)

Batches:   0%|          | 0/284 [00:00<?, ?it/s]

2021-11-26 17:20:18,894 - BERTopic - Transformed documents to Embeddings
2021-11-26 17:20:44,236 - BERTopic - Reduced dimensionality with UMAP
2021-11-26 17:20:44,772 - BERTopic - Clustered UMAP embeddings with HDBSCAN


## Charger\Enregistrer un modèle de Topic Modeling

In [34]:
file_name = "BERTopic_#3_nr-auto_26_11_21"

Enregistrer *topic_model*

In [36]:
topic_model.save(path_to_GGDrive + "BERT_models/" + file_name)

Charger dans *topic_model*

In [ ]:
topic_model = BERTopic.load(path_to_GGDrive + "BERT_models/" + file_name)

# Visualisation des résultats

Aperçu général du Topic Modeling

In [18]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,3415,-1_elle_mais_nous_ce
1,0,482,0_routard_ailleurs_route_rencontres
2,1,427,1_coloriages_colorier_couleurs_couleur
3,2,393,2_elle_son_lui_fille
4,3,179,3_adresses_sites_quartier_boutiques
...,...,...,...
85,84,11,84_salade_salades_recettes_poulet
86,85,11,85_italienne_cuisine_recettes_italiens
87,86,11,86_travail_emploi_psychologie_professionnelle
88,87,10,87_huiles_élixirs_remèdes_thérapeutique


In [55]:
nb_words = 10
for ind in range(0,len(topics)):
  print("TOPIC {0} -> ".format(ind-1) + " ".join([word for [word,_] in topic_model.get_topic(ind-1)][:nb_words]))

TOPIC -1 -> nous comme monde ils être histoire tout notre aussi tous
TOPIC 0 -> elle lui fille mère père amour pourtant son petite pire
TOPIC 1 -> routard ailleurs route rencontres détaillés souvent découvrir découvertes sentiers visites
TOPIC 2 -> cake gâteaux gâteau desserts cakes dessert cupcakes biscuits cookies cheesecake
TOPIC 3 -> crime détective enquête police meurtre singh victime inspecteur assassin mort
TOPIC 4 -> quartier sites bars boutiques restaurants shopping guide hôtels expos festivals
TOPIC 5 -> stickers autocollants pages jeux décorer poster cartonné scènes décors personnages
TOPIC 6 -> enfant parents grossesse bébé enfants parent maternité mère enfance maman
TOPIC 7 -> galaxie will empire guerre skywalker seigneur guerriers rebelles mission kenobi
TOPIC 8 -> plantes jardin fleurs champignons légumes cultiver plante herbes jardiniers jardins
TOPIC 9 -> kitty hello chat chats petit chatons mignon cat amis petits
TOPIC 10 -> motifs colorier coloriages couleurs artiste

TypeError: ignored

Mots prépondérants par topic

In [ ]:
topic_model.get_topic(-1)

[('vie', 0.0026581833630033532),
 ('comme', 0.002652416786649861),
 ('nous', 0.0025990600147941917),
 ('ils', 0.0025627861603095886),
 ('être', 0.0025487377335336975),
 ('sans', 0.0024399602017755977),
 ('notre', 0.002415962417989489),
 ('aussi', 0.0023506693529961975),
 ('tout', 0.0022912650555160954),
 ('même', 0.002259704677301776)]

Intertopic Distance Map

In [20]:
topic_model.visualize_topics()

Clustering hierarchique sur les topics

In [21]:
topic_model.visualize_hierarchy()

Identification de topic par mot-clef

In [22]:
close_topics, similarity = topic_model.find_topics("guerre", top_n=5)
for t,s in zip(close_topics,similarity):
    print([t],s)

[75] 0.9678511258799481
[13] 0.75092867565917
[80] 0.6210636090637133
[35] 0.5975033245361545
[28] 0.587646430639622


In [40]:
topic_model.get_topic(81)

[('univers', 0.035178143447747647),
 ('bang', 0.02442163668103572),
 ('galaxies', 0.02062507623445477),
 ('physiciens', 0.01546880717584108),
 ('cosmologie', 0.014652982008621432),
 ('fluctuations', 0.010312538117227385),
 ('physicien', 0.010013217458187533),
 ('relativité', 0.009561879783309983),
 ('gravité', 0.009224771227601191),
 ('cosmos', 0.008731511965192984)]

Classification d'un nouveau texte

In [52]:
txt_test = """

"""
# OK # Meilleurs restaurants gastronomiques de France, des recettes de cuisine traditionnelles, des ingrédients frais et des légumes bio.
# NO # Retrouvez dans ce magnifique ouvrage illustré les meilleures adresses de la gastronomie française. Découvrez les routes champêtres de l'hexagone au détour d'un petit restaurant traditionnel.
# 1/2 OK # Albert Einstein a révolutionné la physique au début du siècle en redéfinissant la force de gravité comme une distorision de l'espace et du temps. Découvrez l'histoire de ce scientifique hors-normes dans ce magnifique livre illustré présentant des photos d'archive inédites avec des lunettes 3D gracieusement offertes pour l'occasion.
# OK # Présentation de 92 plantes et autres produits naturels et détail de 1.000 formules d'herboristerie pour soigner cent troubles et pathologies du quotidien. Avec un cahier couleur des essences. Tirage limité.
# OK # L'ère de l'intelligence artificielle est déjà là ! Les robots vont tous nous remplacer parce que les ordinateurs sont plus rapides que les humains.

topic_test,proba_test = topic_model.transform(txt_test)
print(topic_test,proba_test)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[4] None


# Analyse des résultats (https://github.com/MaartenGr/BERTopic/issues/90)

In [14]:
def compute_coherence(docs,topics):
    # Preprocess Documents
    documents = pd.DataFrame({"Document": docs,
                              "ID": range(len(docs)),
                              "Topic": topics})
    documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
    cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

    # Extract vectorizer and analyzer from BERTopic
    vectorizer = topic_model.vectorizer_model
    analyzer = vectorizer.build_analyzer()

    # Extract features for Topic Coherence evaluation
    words = vectorizer.get_feature_names()
    tokens = [analyzer(doc) for doc in cleaned_docs]
    dictionary = corpora.Dictionary(tokens)
    corpus = [dictionary.doc2bow(token) for token in tokens]
    topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
                  for topic in range(len(set(topics))-1)]

    # Evaluate
    coherence_model = CoherenceModel(topics=topic_words, 
                                    texts=tokens, 
                                    corpus=corpus,
                                    dictionary=dictionary, 
                                    coherence='c_v')
    
    return(coherence_model.get_coherence())

In [56]:
 coherence = compute_coherence(docs,topics)
 print("Coherence C_v : ",coherence)

Coherence C_v :  0.6633548380605472


# Réduction du nombre de topics

In [ ]:
### PROVISOIRE ###
new_topics, new_probs = topic_model.reduce_topics(docs, topics, probabilities=probs, nr_topics=30)
for ind in range(0,len(topics)):
  print("TOPIC {0} -> ".format(ind-1) + " ".join([word for [word,_] in topic_model.get_topic(ind-1)][:nb_words]))